# Module 6 & 7 - Associations

## Notes
- Install these packages:

    + `pip3 install mlxtend`
    + `pip3 install openpyxl`

- Training data set is from here: 
    
    + `https://archive.ics.uci.edu/ml/datasets/online+retail`  
    + `http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx`

- Code example is from here: 
    + `https://pbpython.com/market-basket-analysis.html`
    
- Support & Confidence:

    Support, $s(X \to Y) = \frac{\sigma(X \cup Y)}{N}$
    
    Confidence, $c(X \to Y) = \frac{\sigma(X \cup Y)}{\sigma (X)}$
    
    Where, $X \cap Y = \emptyset $

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import apriori

from itertools import permutations, combinations

In [ ]:
df = pd.read_excel('../data/Online_Retail.xlsx')
# df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
init_series = df.copy()
df.head()

## Notes
Remove items that lack invoice number (Null/NaN) or have letter `C` as `credit transactions`.


In [ ]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
print('initial length:', len(init_series))
print('    new length:', len(df))
print(' items removed:', len(init_series)-len(df))

In [ ]:
basket = (df[df['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

In [ ]:
len(rules)

In [ ]:
new_rules = rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]
new_rules

In [ ]:
len(new_rules)

In [ ]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

In [ ]:
basket['ALARM CLOCK BAKELIKE RED'].sum()


In [ ]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]


# Example #2

`https://medium.com/analytics-vidhya/association-analysis-in-python-2b955d0180c`



In [50]:

df = pd.read_csv('../data/retail_dataset.csv')
dataset = df.to_numpy()
df = df.replace(np.nan, '', regex=True)
df

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,,
3,Cheese,Meat,Eggs,Milk,Wine,,
4,Meat,Pencil,Wine,,,,
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,,,,
311,Meat,Milk,Pencil,,,,
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,,,,,


In [51]:

dataset = df.to_numpy()


new_dataset = []
for row in dataset:
    new_row = []
    for i in row:
        if i != '':
            new_row.append(i)
    new_dataset.append(new_row)

te = TransactionEncoder()
te_ary = te.fit(new_dataset).transform(new_dataset)
df1 = pd.DataFrame(te_ary, columns=te.columns_)    
df1    

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,False,True,True,True,True,True,False,True,True
1,False,True,True,True,False,True,True,True,True
2,False,False,True,False,True,True,True,False,True
3,False,False,True,False,True,True,True,False,True
4,False,False,False,False,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...
310,False,True,True,False,True,False,False,False,False
311,False,False,False,False,False,True,True,True,False
312,False,True,True,True,True,True,False,True,True
313,False,False,True,False,False,True,False,False,False


In [ ]:
freq_items = apriori(df1, min_support=0.25, use_colnames=True, verbose=1)
freq_items

In [ ]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
rules.head()
print(len(rules))

In [ ]:
plt.scatter(rules['support'], rules['confidence'], alpha=0.5)
plt.xlabel('support')
plt.ylabel('confidence')
plt.title('Support vs Confidence')
plt.show()

In [ ]:
plt.scatter(rules['support'], rules['lift'], alpha=0.5)
plt.xlabel('support')
plt.ylabel('lift')
plt.title('Support vs Lift')
plt.show()

In [ ]:
fit = np.polyfit(rules['lift'], rules['confidence'], 1)
plt.xlabel('lift')
plt.ylabel('confidence')
fit_fn = np.poly1d(fit)
plt.plot(rules['lift'], rules['confidence'], 'yo', rules['lift'], fit_fn(rules['lift']))

# Homework Assignment # 3

1. Run the entire code below with these two datasets:

```python
T = [['a','b'], ['c','d'], ['a','c'], ['a','b','d'], ['a','e'], ['d'], ['a'], ['b']]
T = [['Bread', 'Milk'], 
     ['Bread', 'Diapers', 'Beer', 'Eggs'], 
     ['Milk', 'Diapers', 'Beer', 'Cola'],
     ['Bread', 'Milk', 'Diapers', 'Beer'],
     ['Bread', 'Milk', 'Diapers', 'Cola']]

```
2. Don't run the congressional vote dataset. It is too large (36 variables)

3. Verify the results of the program manually using the algorithms provided. 

4. Write your report and submit. Use either Jupyter notebook or Word or both.


In [ ]:
issues = ['','handicapped-infants','water-project-cost-sharing',
          'adoption-of-the-budget-resolution',
          'physician-fee-freeze',
          'el-salvador-aid',
          'religious-groups-in-schools',
          'anti-satellite-test-ban',
          'aid-to-nicaraguan-contras',
          'mx-missile',
          'immigration',
          'synfuels-corporation-cutback',
          'education-spending',
          'superfund-right-to-sue',
          'crime',
          'duty-free-exports',
          'export-administration-act-south-africa']
len(issues)

## Loading congressional votes 1984.

In [ ]:
df = pd.read_csv('../data/house-votes-84.data')

## Making Transaction set from a dataframe

In [53]:
def make_T_from_dataframe(df):
    new_T = []
    for index, row in df.iterrows():
        t = []
        for item in row:
            if len(item) > 0:
                t.append(item)
        new_T.append(t)
    return new_T

In [54]:
def get_84_congressional_votes(df):
    new_T = []
    for index, row in df.iterrows():
        t = []
        j = 0
        for item in row:
            if item == 'y' or item == 'n':
                # t.append(issues[j]+'--'+item)
                t.append(f'{j}_{item}')
            elif item != '?':
                t.append(item)
            j += 1
        new_T.append(t)
    return new_T


## Loading a Transaction sets

In [55]:
T = [['a','b'], ['c','d'], ['a','c'], ['a','b','d'], ['a','e'], ['d'], ['a'], ['b']]
T = [['Bread', 'Milk'], 
     ['Bread', 'Diapers', 'Beer', 'Eggs'], 
     ['Milk', 'Diapers', 'Beer', 'Cola'],
     ['Bread', 'Milk', 'Diapers', 'Beer'],
     ['Bread', 'Milk', 'Diapers', 'Cola']]

# df is loading elsewhere above
T = make_T_from_dataframe(df) 

# T = get_84_congressional_votes(df)

sorted_T = []
for t in T:
    t.sort()
    sorted_T.append(t)
T = sorted_T    
list(T)
print(f'There are {len(T)} transactions:')
display(T)

There are 315 transactions:


[['Bread', 'Cheese', 'Diaper', 'Eggs', 'Meat', 'Pencil', 'Wine'],
 ['Bread', 'Cheese', 'Diaper', 'Meat', 'Milk', 'Pencil', 'Wine'],
 ['Cheese', 'Eggs', 'Meat', 'Milk', 'Wine'],
 ['Cheese', 'Eggs', 'Meat', 'Milk', 'Wine'],
 ['Meat', 'Pencil', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Milk', 'Pencil', 'Wine'],
 ['Cheese', 'Eggs', 'Pencil', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Milk', 'Pencil'],
 ['Bread', 'Cheese', 'Diaper', 'Eggs', 'Milk', 'Wine'],
 ['Bagel', 'Cheese', 'Diaper', 'Eggs', 'Meat', 'Pencil', 'Wine'],
 ['Cheese', 'Eggs', 'Meat', 'Milk', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Meat', 'Milk', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Meat', 'Pencil'],
 ['Bagel', 'Cheese', 'Meat', 'Milk'],
 ['Bread'],
 ['Bagel', 'Diaper', 'Pencil'],
 ['Bagel', 'Bread', 'Meat'],
 ['Bagel', 'Bread', 'Milk'],
 ['Diaper'],
 ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Eggs', 'Meat'],
 ['Bread', 'Cheese', 'Meat', 'Pencil'],
 ['Bread', 'Cheese', 'Eggs', 'Meat', 'Milk', 'Pencil', 'Wine'],
 ['

In [56]:
# Making Universe set U from transaction set T
def make_U(T):    
    u = {}
    for t in T:
        for i in t:
            u[i] = i
    
    U = list(u)
    U.sort()
    return U

U = make_U(T)

print(f'There are {len(U)} items:')
display(U)

There are 9 items:


['Bagel',
 'Bread',
 'Cheese',
 'Diaper',
 'Eggs',
 'Meat',
 'Milk',
 'Pencil',
 'Wine']

# Brute-force
Printing out a lattice of n items of universe U

In [4]:
"""
Generate all possible k-itemsets for a universe U
Organize the list by row, with row 0 contains null,
row 1 contains all 1-itemsets, row 2 contains all 2-itemsets, etc.
"""
def bf_all_itemsets(U):
    bf_candidates = [[]]
    print(U)
    for c_len in range(1,len(U)+1):
        cs = combinations(U,c_len)
        row = []
        for c in cs:
            c = list(c)
            c.sort()
            row.append(c)

        bf_candidates.append(row)
    return bf_candidates


## Check to see if an item is in an itemset

In [5]:
"""
return whether or not a k_itemset is in a transaction t.
"""
def is_k_itemset_in_transaction(k_itemset, t):
    return all(item in t for item in k_itemset)

## Get all the FI candidates that meet or exceed the support count based on a set of transaction

In [6]:
"""
For a given set of k-itemsets, with at support count against a transaction T,
return a set of frequent k-itemsets.
"""

def get_candidates(sup_count, k_itemsets, T):
    k_itemset_coll = {}
    
    """
    First, we count the occurence of a k-itemset in the set of 
    transaction T.
    """
    for k_itemset in k_itemsets:
        k_itemset_key = '_'.join(k_itemset)
        for t in T:
            if is_k_itemset_in_transaction(k_itemset, t):
                if k_itemset_key in k_itemset_coll:
                    item = k_itemset_coll[k_itemset_key]
                    item[1] += 1
                    k_itemset_coll[k_itemset_key] = item
                else:
                    item = [k_itemset, 1]
                    k_itemset_coll[k_itemset_key] = item           

    """
    Then, we are looking for k-itemsets that is infrequent.
    We keep the keys of the removal k-itemsets in a list
    We then go through the removal list and remove the 
    k-itemsets by key
    """                
    remove_keys = []
    for i in k_itemset_coll:
        item = k_itemset_coll[i]
        if item[1] < sup_count:
            remove_keys.append(i)
    for key in remove_keys:
        del k_itemset_coll[key]
        
    return k_itemset_coll



## Displaying all itemsets

In [8]:
all_itemsets = bf_all_itemsets(U)
# display(all_itemsets)

['a', 'b', 'c', 'd', 'e']


### Displaying the candidates and support count

In [9]:
for sup_count in range(1, len(T)+1):
    
    minsup = sup_count/len(T)
    print(f'\n*** supp_count:{sup_count}, minsup:{minsup}\n')

    candidates = [[]] # Null first entry as in lattice      

    for i in range(1,len(all_itemsets)):
        candidates.append(get_candidates(sup_count, all_itemsets[i], T))

    for i in range(1,len(candidates)):
        print(f'{i}({len(candidates[i])}): {candidates[i]}\n')


*** supp_count:1, minsup:0.125

1(5): {'a': [['a'], 5], 'b': [['b'], 3], 'c': [['c'], 2], 'd': [['d'], 3], 'e': [['e'], 1]}

2(6): {'a_b': [['a', 'b'], 2], 'a_c': [['a', 'c'], 1], 'a_d': [['a', 'd'], 1], 'a_e': [['a', 'e'], 1], 'b_d': [['b', 'd'], 1], 'c_d': [['c', 'd'], 1]}

3(1): {'a_b_d': [['a', 'b', 'd'], 1]}

4(0): {}

5(0): {}


*** supp_count:2, minsup:0.25

1(4): {'a': [['a'], 5], 'b': [['b'], 3], 'c': [['c'], 2], 'd': [['d'], 3]}

2(1): {'a_b': [['a', 'b'], 2]}

3(0): {}

4(0): {}

5(0): {}


*** supp_count:3, minsup:0.375

1(3): {'a': [['a'], 5], 'b': [['b'], 3], 'd': [['d'], 3]}

2(0): {}

3(0): {}

4(0): {}

5(0): {}


*** supp_count:4, minsup:0.5

1(1): {'a': [['a'], 5]}

2(0): {}

3(0): {}

4(0): {}

5(0): {}


*** supp_count:5, minsup:0.625

1(1): {'a': [['a'], 5]}

2(0): {}

3(0): {}

4(0): {}

5(0): {}


*** supp_count:6, minsup:0.75

1(0): {}

2(0): {}

3(0): {}

4(0): {}

5(0): {}


*** supp_count:7, minsup:0.875

1(0): {}

2(0): {}

3(0): {}

4(0): {}

5(0): {}


*

# Apriori algorithm
Implement Apriori algorithm ...

$F_{k} = {i\ |\ i \in I \land \sigma({i}) \ge N \ x \ minsup}$

## Step 1: Generating `k` itemsets from Universal U

In [10]:
"""
Generating a set of all possible k-itemsets transactions T 
from a set of universal items U
"""
def make_T(a_U, k):
    cs = combinations(a_U, k)
    k_itemsets = []
    for c in cs:
        c = list(c)
        c.sort()
        k_itemsets.append(c)
    return k_itemsets

### Extract items out of a collection of k-itemsets

In [11]:
def get_k_itemsets(ck):
    k_itemsets = []
    for item in ck:
        k_itemsets.append(ck[item][0])
    return k_itemsets

### Prune

In [12]:
"""
Pruning. starting with 2. If k-1 subset of k_candidate 
is not a part of k_minus_1_candidate, drop the k-itemset.
"""
def candidate_prune(kminus1_candidates, k_candidates):
    print('(k-1)-itemsets:')
    display(kminus1_candidates)
    print('k-itemsets:')
    display(k_candidates)
    
    return_k_candidates = []
    
    for i in range(len(k_candidates)):        
        k = len(k_candidates[i])        
        if k == 1:
            print('*** Nothing to prune ***')
            return k_candidates
            
        k_candidate = [k_candidates[i]]
        a_U = make_U(k_candidate)
        kminus1_subsets = make_T(a_U, k-1)
        if all(item in kminus1_candidates for item in kminus1_subsets):
            return_k_candidates.append(k_candidates[i])
            
    return return_k_candidates

### Generating Candidates

In [13]:
"""
Generating all possible k-itemset transactions T
from a previous set of (k-1)-itemsets 
"""
def candidate_gen(kminus1_itemsets, k):  
    return make_T(make_U(kminus1_itemsets), k)

### The apriori algorithm.

In [29]:
"""
"""    
def apriori_gen(U, T, sup_count):    
    UFk = [[]]
    a_U = U
    # display(a_U)
    a_T = make_T(a_U, 1)
    display(a_T)

    prev_ck = a_T
    for k in range (1, 6):
        print(f'*** generating {k}-itemsets ***')
        #k_itemsets = make_T(a_U, k)
        k_itemsets = candidate_gen(prev_ck, k)
        
        print(f'*** pruning {k}-itemsets ***')
        print('** before pruning **')
        display(k_itemsets)
        k_itemsets = candidate_prune(prev_ck, k_itemsets)        
        print('** after pruning **')
        display(k_itemsets)

        print('*** verifying frequent itemsets ***')
        ck = get_candidates(sup_count, k_itemsets, T)
        display(ck)
        # k_itemsets = get_k_itemsets(ck)
        prev_ck = get_k_itemsets(ck)

        if ck == None or len(ck) == 0:
            print ('** DONE **')
            break

        UFk.append(ck)
        k_itemsets = get_k_itemsets(ck)
        a_U = make_U(k_itemsets)
        print('a_U:', a_U)
    return UFk


In [58]:
sup_count = 4
UFk = apriori_gen(U, T, sup_count)

print('*** UFk ***')
display(UFk)

[['Bagel'],
 ['Bread'],
 ['Cheese'],
 ['Diaper'],
 ['Eggs'],
 ['Meat'],
 ['Milk'],
 ['Pencil'],
 ['Wine']]

*** generating 1-itemsets ***
*** pruning 1-itemsets ***
** before pruning **


[['Bagel'],
 ['Bread'],
 ['Cheese'],
 ['Diaper'],
 ['Eggs'],
 ['Meat'],
 ['Milk'],
 ['Pencil'],
 ['Wine']]

(k-1)-itemsets:


[['Bagel'],
 ['Bread'],
 ['Cheese'],
 ['Diaper'],
 ['Eggs'],
 ['Meat'],
 ['Milk'],
 ['Pencil'],
 ['Wine']]

...

[['Bagel', 'Bread', 'Cheese', 'Diaper', 'Eggs'],
 ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Meat'],
 ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Milk'],
 ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Pencil'],
 ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Wine'],
 ['Bagel', 'Bread', 'Cheese', 'Eggs', 'Meat'],
 ['Bagel', 'Bread', 'Cheese', 'Eggs', 'Milk'],
 ['Bagel', 'Bread', 'Cheese', 'Eggs', 'Pencil'],
 ['Bagel', 'Bread', 'Cheese', 'Eggs', 'Wine'],
 ['Bagel', 'Bread', 'Cheese', 'Meat', 'Milk'],
 ['Bagel', 'Bread', 'Cheese', 'Meat', 'Pencil'],
 ['Bagel', 'Bread', 'Cheese', 'Meat', 'Wine'],
 ['Bagel', 'Bread', 'Cheese', 'Milk', 'Pencil'],
 ['Bagel', 'Bread', 'Cheese', 'Milk', 'Wine'],
 ['Bagel', 'Bread', 'Cheese', 'Pencil', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Meat'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Milk'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Pencil'],
 ['Bagel', 'Bread', 'Diaper', 'Eggs', 'Wine'],
 ['Bagel', 'Bread', 'Diaper', 'Meat', 'Milk'],
 ['Bagel', 'Bread', 'Diaper', 'Meat', 

*** verifying frequent itemsets ***


{'Bagel_Bread_Cheese_Diaper_Eggs': [['Bagel',
   'Bread',
   'Cheese',
   'Diaper',
   'Eggs'],
  7],
 'Bagel_Bread_Cheese_Diaper_Meat': [['Bagel',
   'Bread',
   'Cheese',
   'Diaper',
   'Meat'],
  13],
 'Bagel_Bread_Cheese_Diaper_Milk': [['Bagel',
   'Bread',
   'Cheese',
   'Diaper',
   'Milk'],
  7],
 'Bagel_Bread_Cheese_Diaper_Pencil': [['Bagel',
   'Bread',
   'Cheese',
   'Diaper',
   'Pencil'],
  8],
 'Bagel_Bread_Cheese_Diaper_Wine': [['Bagel',
   'Bread',
   'Cheese',
   'Diaper',
   'Wine'],
  11],
 'Bagel_Bread_Cheese_Eggs_Meat': [['Bagel', 'Bread', 'Cheese', 'Eggs', 'Meat'],
  6],
 'Bagel_Bread_Cheese_Eggs_Milk': [['Bagel', 'Bread', 'Cheese', 'Eggs', 'Milk'],
  5],
 'Bagel_Bread_Cheese_Eggs_Pencil': [['Bagel',
   'Bread',
   'Cheese',
   'Eggs',
   'Pencil'],
  8],
 'Bagel_Bread_Cheese_Eggs_Wine': [['Bagel', 'Bread', 'Cheese', 'Eggs', 'Wine'],
  8],
 'Bagel_Bread_Cheese_Meat_Milk': [['Bagel', 'Bread', 'Cheese', 'Meat', 'Milk'],
  9],
 'Bagel_Bread_Cheese_Meat_Pencil': [['

a_U: ['Bagel', 'Bread', 'Cheese', 'Diaper', 'Eggs', 'Meat', 'Milk', 'Pencil', 'Wine']
*** UFk ***


[[],
 {'Bagel': [['Bagel'], 134],
  'Bread': [['Bread'], 159],
  'Cheese': [['Cheese'], 158],
  'Diaper': [['Diaper'], 128],
  'Eggs': [['Eggs'], 138],
  'Meat': [['Meat'], 150],
  'Milk': [['Milk'], 158],
  'Pencil': [['Pencil'], 114],
  'Wine': [['Wine'], 138]},
 {'Bagel_Bread': [['Bagel', 'Bread'], 88],
  'Bagel_Cheese': [['Bagel', 'Cheese'], 61],
  'Bagel_Diaper': [['Bagel', 'Diaper'], 58],
  'Bagel_Eggs': [['Bagel', 'Eggs'], 48],
  'Bagel_Meat': [['Bagel', 'Meat'], 60],
  'Bagel_Milk': [['Bagel', 'Milk'], 71],
  'Bagel_Pencil': [['Bagel', 'Pencil'], 50],
  'Bagel_Wine': [['Bagel', 'Wine'], 54],
  'Bread_Cheese': [['Bread', 'Cheese'], 75],
  'Bread_Diaper': [['Bread', 'Diaper'], 73],
  'Bread_Eggs': [['Bread', 'Eggs'], 59],
  'Bread_Meat': [['Bread', 'Meat'], 65],
  'Bread_Milk': [['Bread', 'Milk'], 88],
  'Bread_Pencil': [['Bread', 'Pencil'], 63],
  'Bread_Wine': [['Bread', 'Wine'], 77],
  'Cheese_Diaper': [['Cheese', 'Diaper'], 63],
  'Cheese_Eggs': [['Cheese', 'Eggs'], 94],
  'C

In [39]:
# Computing Confidence
# for each frequent k-itemset fk, k >= 2 do
#   H1 = 1
# end for

for k in range(1, len(UFk)):
    for key in UFk[k]:
        k_itemset = UFk[k][key][0]
        k_count = UFk[k][key][1]
        print(key, k_itemset, k_count)
        h1 = []
        for item in k_itemset:
            h1.append([item])
        print(h1)


        

Beer ['Beer'] 3
[['Beer']]
Bread ['Bread'] 4
[['Bread']]
Cola ['Cola'] 2
[['Cola']]
Diapers ['Diapers'] 4
[['Diapers']]
Milk ['Milk'] 4
[['Milk']]
Beer_Bread ['Beer', 'Bread'] 2
[['Beer'], ['Bread']]
Beer_Diapers ['Beer', 'Diapers'] 3
[['Beer'], ['Diapers']]
Beer_Milk ['Beer', 'Milk'] 2
[['Beer'], ['Milk']]
Bread_Diapers ['Bread', 'Diapers'] 3
[['Bread'], ['Diapers']]
Bread_Milk ['Bread', 'Milk'] 3
[['Bread'], ['Milk']]
Cola_Diapers ['Cola', 'Diapers'] 2
[['Cola'], ['Diapers']]
Cola_Milk ['Cola', 'Milk'] 2
[['Cola'], ['Milk']]
Diapers_Milk ['Diapers', 'Milk'] 3
[['Diapers'], ['Milk']]
Beer_Bread_Diapers ['Beer', 'Bread', 'Diapers'] 2
[['Beer'], ['Bread'], ['Diapers']]
Beer_Diapers_Milk ['Beer', 'Diapers', 'Milk'] 2
[['Beer'], ['Diapers'], ['Milk']]
Bread_Diapers_Milk ['Bread', 'Diapers', 'Milk'] 2
[['Bread'], ['Diapers'], ['Milk']]
Cola_Diapers_Milk ['Cola', 'Diapers', 'Milk'] 2
[['Cola'], ['Diapers'], ['Milk']]


In [59]:
# itemset = ['a','b','c']
all_rules = {}
for k in range(2, len(UFk)):
    for key in UFk[k]:
        k_itemset = UFk[k][key][0]
        k_count = UFk[k][key][1]
        print('\n\n***', key, k_itemset, k_count)
        h1 = []

        super_set = []
        for count in range(1,len(k_itemset)):
            for c in combinations(k_itemset, count):
                value_key = '_'.join(c)
                print(key, value_key, c, UFk[len(c)][value_key][1])
                super_set.append(c)
        
        sig_a_b = k_count
        all_rules = {}
        for i in range(len(super_set)-1):
            a = super_set[i]
            for j in range(i+1, len(super_set)):
                b = super_set[j]
                if set(a).isdisjoint(b) and len(a) + len(b) == len(k_itemset):
                    sig_a = UFk[len(a)]['_'.join(a)][1]
                    sig_b = UFk[len(b)]['_'.join(b)][1]                                        
                    all_rules['_'.join(a) + '->' + '_'.join(b)] = sig_a_b/sig_a
                    all_rules['_'.join(b) + '->' + '_'.join(a)] = sig_a_b/sig_b
                    



*** Bagel_Bread ['Bagel', 'Bread'] 88
Bagel_Bread Bagel ('Bagel',) 134
Bagel_Bread Bread ('Bread',) 159


*** Bagel_Cheese ['Bagel', 'Cheese'] 61
Bagel_Cheese Bagel ('Bagel',) 134
Bagel_Cheese Cheese ('Cheese',) 158


*** Bagel_Diaper ['Bagel', 'Diaper'] 58
Bagel_Diaper Bagel ('Bagel',) 134
Bagel_Diaper Diaper ('Diaper',) 128


*** Bagel_Eggs ['Bagel', 'Eggs'] 48
Bagel_Eggs Bagel ('Bagel',) 134
Bagel_Eggs Eggs ('Eggs',) 138


*** Bagel_Meat ['Bagel', 'Meat'] 60
Bagel_Meat Bagel ('Bagel',) 134
Bagel_Meat Meat ('Meat',) 150


*** Bagel_Milk ['Bagel', 'Milk'] 71
Bagel_Milk Bagel ('Bagel',) 134
Bagel_Milk Milk ('Milk',) 158


*** Bagel_Pencil ['Bagel', 'Pencil'] 50
Bagel_Pencil Bagel ('Bagel',) 134
Bagel_Pencil Pencil ('Pencil',) 114


*** Bagel_Wine ['Bagel', 'Wine'] 54
Bagel_Wine Bagel ('Bagel',) 134
Bagel_Wine Wine ('Wine',) 138


*** Bread_Cheese ['Bread', 'Cheese'] 75
Bread_Cheese Bread ('Bread',) 159
Bread_Cheese Cheese ('Cheese',) 158


*** Bread_Diaper ['Bread', 'Diaper'] 73
Brea

*** Bagel_Bread_Meat_Wine ['Bagel', 'Bread', 'Meat', 'Wine'] 19
Bagel_Bread_Meat_Wine Bagel ('Bagel',) 134
Bagel_Bread_Meat_Wine Bread ('Bread',) 159
Bagel_Bread_Meat_Wine Meat ('Meat',) 150
Bagel_Bread_Meat_Wine Wine ('Wine',) 138
Bagel_Bread_Meat_Wine Bagel_Bread ('Bagel', 'Bread') 88
Bagel_Bread_Meat_Wine Bagel_Meat ('Bagel', 'Meat') 60
Bagel_Bread_Meat_Wine Bagel_Wine ('Bagel', 'Wine') 54
Bagel_Bread_Meat_Wine Bread_Meat ('Bread', 'Meat') 65
Bagel_Bread_Meat_Wine Bread_Wine ('Bread', 'Wine') 77
Bagel_Bread_Meat_Wine Meat_Wine ('Meat', 'Wine') 79
Bagel_Bread_Meat_Wine Bagel_Bread_Meat ('Bagel', 'Bread', 'Meat') 36
Bagel_Bread_Meat_Wine Bagel_Bread_Wine ('Bagel', 'Bread', 'Wine') 32
Bagel_Bread_Meat_Wine Bagel_Meat_Wine ('Bagel', 'Meat', 'Wine') 33
Bagel_Bread_Meat_Wine Bread_Meat_Wine ('Bread', 'Meat', 'Wine') 42


*** Bagel_Bread_Milk_Pencil ['Bagel', 'Bread', 'Milk', 'Pencil'] 14
Bagel_Bread_Milk_Pencil Bagel ('Bagel',) 134
Bagel_Bread_Milk_Pencil Bread ('Bread',) 159
Bagel_Bread_

Cheese_Diaper_Meat_Wine Meat ('Meat',) 150
Cheese_Diaper_Meat_Wine Wine ('Wine',) 138
Cheese_Diaper_Meat_Wine Cheese_Diaper ('Cheese', 'Diaper') 63
Cheese_Diaper_Meat_Wine Cheese_Meat ('Cheese', 'Meat') 102
Cheese_Diaper_Meat_Wine Cheese_Wine ('Cheese', 'Wine') 85
Cheese_Diaper_Meat_Wine Diaper_Meat ('Diaper', 'Meat') 61
Cheese_Diaper_Meat_Wine Diaper_Wine ('Diaper', 'Wine') 74
Cheese_Diaper_Meat_Wine Meat_Wine ('Meat', 'Wine') 79
Cheese_Diaper_Meat_Wine Cheese_Diaper_Meat ('Cheese', 'Diaper', 'Meat') 36
Cheese_Diaper_Meat_Wine Cheese_Diaper_Wine ('Cheese', 'Diaper', 'Wine') 43
Cheese_Diaper_Meat_Wine Cheese_Meat_Wine ('Cheese', 'Meat', 'Wine') 53
Cheese_Diaper_Meat_Wine Diaper_Meat_Wine ('Diaper', 'Meat', 'Wine') 39


*** Cheese_Diaper_Milk_Pencil ['Cheese', 'Diaper', 'Milk', 'Pencil'] 11
Cheese_Diaper_Milk_Pencil Cheese ('Cheese',) 158
Cheese_Diaper_Milk_Pencil Diaper ('Diaper',) 128
Cheese_Diaper_Milk_Pencil Milk ('Milk',) 158
Cheese_Diaper_Milk_Pencil Pencil ('Pencil',) 114
Cheese_

Bagel_Bread_Diaper_Milk_Pencil Bagel_Bread_Milk ('Bagel', 'Bread', 'Milk') 54
Bagel_Bread_Diaper_Milk_Pencil Bagel_Bread_Pencil ('Bagel', 'Bread', 'Pencil') 28
Bagel_Bread_Diaper_Milk_Pencil Bagel_Diaper_Milk ('Bagel', 'Diaper', 'Milk') 25
Bagel_Bread_Diaper_Milk_Pencil Bagel_Diaper_Pencil ('Bagel', 'Diaper', 'Pencil') 26
Bagel_Bread_Diaper_Milk_Pencil Bagel_Milk_Pencil ('Bagel', 'Milk', 'Pencil') 22
Bagel_Bread_Diaper_Milk_Pencil Bread_Diaper_Milk ('Bread', 'Diaper', 'Milk') 36
Bagel_Bread_Diaper_Milk_Pencil Bread_Diaper_Pencil ('Bread', 'Diaper', 'Pencil') 33
Bagel_Bread_Diaper_Milk_Pencil Bread_Milk_Pencil ('Bread', 'Milk', 'Pencil') 34
Bagel_Bread_Diaper_Milk_Pencil Diaper_Milk_Pencil ('Diaper', 'Milk', 'Pencil') 23
Bagel_Bread_Diaper_Milk_Pencil Bagel_Bread_Diaper_Milk ('Bagel', 'Bread', 'Diaper', 'Milk') 18
Bagel_Bread_Diaper_Milk_Pencil Bagel_Bread_Diaper_Pencil ('Bagel', 'Bread', 'Diaper', 'Pencil') 15
Bagel_Bread_Diaper_Milk_Pencil Bagel_Bread_Milk_Pencil ('Bagel', 'Bread', 'M

Bagel_Diaper_Eggs_Meat_Pencil Bagel_Eggs_Meat_Pencil ('Bagel', 'Eggs', 'Meat', 'Pencil') 12
Bagel_Diaper_Eggs_Meat_Pencil Diaper_Eggs_Meat_Pencil ('Diaper', 'Eggs', 'Meat', 'Pencil') 11


*** Bagel_Diaper_Eggs_Meat_Wine ['Bagel', 'Diaper', 'Eggs', 'Meat', 'Wine'] 9
Bagel_Diaper_Eggs_Meat_Wine Bagel ('Bagel',) 134
Bagel_Diaper_Eggs_Meat_Wine Diaper ('Diaper',) 128
Bagel_Diaper_Eggs_Meat_Wine Eggs ('Eggs',) 138
Bagel_Diaper_Eggs_Meat_Wine Meat ('Meat',) 150
Bagel_Diaper_Eggs_Meat_Wine Wine ('Wine',) 138
Bagel_Diaper_Eggs_Meat_Wine Bagel_Diaper ('Bagel', 'Diaper') 58
Bagel_Diaper_Eggs_Meat_Wine Bagel_Eggs ('Bagel', 'Eggs') 48
Bagel_Diaper_Eggs_Meat_Wine Bagel_Meat ('Bagel', 'Meat') 60
Bagel_Diaper_Eggs_Meat_Wine Bagel_Wine ('Bagel', 'Wine') 54
Bagel_Diaper_Eggs_Meat_Wine Diaper_Eggs ('Diaper', 'Eggs') 51
Bagel_Diaper_Eggs_Meat_Wine Diaper_Meat ('Diaper', 'Meat') 61
Bagel_Diaper_Eggs_Meat_Wine Diaper_Wine ('Diaper', 'Wine') 74
Bagel_Diaper_Eggs_Meat_Wine Eggs_Meat ('Eggs', 'Meat') 84
Bagel

Bread_Diaper_Eggs_Meat_Pencil Bread_Eggs ('Bread', 'Eggs') 59
Bread_Diaper_Eggs_Meat_Pencil Bread_Meat ('Bread', 'Meat') 65
Bread_Diaper_Eggs_Meat_Pencil Bread_Pencil ('Bread', 'Pencil') 63
Bread_Diaper_Eggs_Meat_Pencil Diaper_Eggs ('Diaper', 'Eggs') 51
Bread_Diaper_Eggs_Meat_Pencil Diaper_Meat ('Diaper', 'Meat') 61
Bread_Diaper_Eggs_Meat_Pencil Diaper_Pencil ('Diaper', 'Pencil') 54
Bread_Diaper_Eggs_Meat_Pencil Eggs_Meat ('Eggs', 'Meat') 84
Bread_Diaper_Eggs_Meat_Pencil Eggs_Pencil ('Eggs', 'Pencil') 52
Bread_Diaper_Eggs_Meat_Pencil Meat_Pencil ('Meat', 'Pencil') 56
Bread_Diaper_Eggs_Meat_Pencil Bread_Diaper_Eggs ('Bread', 'Diaper', 'Eggs') 28
Bread_Diaper_Eggs_Meat_Pencil Bread_Diaper_Meat ('Bread', 'Diaper', 'Meat') 38
Bread_Diaper_Eggs_Meat_Pencil Bread_Diaper_Pencil ('Bread', 'Diaper', 'Pencil') 33
Bread_Diaper_Eggs_Meat_Pencil Bread_Eggs_Meat ('Bread', 'Eggs', 'Meat') 29
Bread_Diaper_Eggs_Meat_Pencil Bread_Eggs_Pencil ('Bread', 'Eggs', 'Pencil') 31
Bread_Diaper_Eggs_Meat_Pencil B

In [60]:
print(f'there are {len(all_rules)} rules')
all_rules

there are 30 rules


{'Eggs->Meat_Milk_Pencil_Wine': 0.06521739130434782,
 'Meat_Milk_Pencil_Wine->Eggs': 0.5625,
 'Meat->Eggs_Milk_Pencil_Wine': 0.06,
 'Eggs_Milk_Pencil_Wine->Meat': 0.5,
 'Milk->Eggs_Meat_Pencil_Wine': 0.056962025316455694,
 'Eggs_Meat_Pencil_Wine->Milk': 0.47368421052631576,
 'Pencil->Eggs_Meat_Milk_Wine': 0.07894736842105263,
 'Eggs_Meat_Milk_Wine->Pencil': 0.3103448275862069,
 'Wine->Eggs_Meat_Milk_Pencil': 0.06521739130434782,
 'Eggs_Meat_Milk_Pencil->Wine': 0.6923076923076923,
 'Eggs_Meat->Milk_Pencil_Wine': 0.10714285714285714,
 'Milk_Pencil_Wine->Eggs_Meat': 0.3,
 'Eggs_Milk->Meat_Pencil_Wine': 0.11688311688311688,
 'Meat_Pencil_Wine->Eggs_Milk': 0.25,
 'Eggs_Pencil->Meat_Milk_Wine': 0.17307692307692307,
 'Meat_Milk_Wine->Eggs_Pencil': 0.225,
 'Eggs_Wine->Meat_Milk_Pencil': 0.11842105263157894,
 'Meat_Milk_Pencil->Eggs_Wine': 0.36,
 'Meat_Milk->Eggs_Pencil_Wine': 0.11688311688311688,
 'Eggs_Pencil_Wine->Meat_Milk': 0.23684210526315788,
 'Meat_Pencil->Eggs_Milk_Wine': 0.16071428571

In [61]:
print(f'number of transaction in the dataset is {len(T)}')

number of transaction in the dataset is 315


In [65]:
confidence = 0.5
adopted_rules = {}
for key in all_rules:
    if all_rules[key] >= confidence:
        adopted_rules[key] = all_rules[key]

In [66]:
print(f'there are {len(adopted_rules)} adopted rules of a {confidence} of confidence or greater')

for key in adopted_rules:
    if key.startswith('adoption-of-the-budget-resolution_mx-missile') and key.endswith('->democrat'):
        print(key, adopted_rules[key])

there are 3 adopted rules of a 0.5 of confidence or greater


In [67]:
adopted_rules

{'Meat_Milk_Pencil_Wine->Eggs': 0.5625,
 'Eggs_Milk_Pencil_Wine->Meat': 0.5,
 'Eggs_Meat_Milk_Pencil->Wine': 0.6923076923076923}

## Congressional Voting Record examples
